In [ ]:
!python --version

Python 3.9.16


# kobert task

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! nvidia-smi

Mon Apr  3 12:27:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/drive/MyDrive/졸플/BERT

/content/drive/MyDrive/졸플/BERT


In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3a_ksbdo
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3a_ksbdo
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.6 MB/s eta 0:00

In [ ]:
!python --version

Python 3.9.16


In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import gluonnlp as nlp
from tqdm.notebook import tqdm, tqdm_notebook

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

In [ ]:
torch.cuda.get_arch_list()

['sm_37', 'sm_50', 'sm_60', 'sm_70']

In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
torch.rand(10).cuda()

tensor([0.2393, 0.2107, 0.5106, 0.5237, 0.5495, 0.8076, 0.4382, 0.2457, 0.5109,
        0.8180], device='cuda:0')

In [ ]:
from transformers import AutoModel, AutoTokenizer
kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/졸플/BERT/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/졸플/BERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# kobert_tokenizer = get_tokenizer()

In [ ]:
tok=kobert_tokenizer.tokenize

### data load

In [ ]:
regular_df = pd.read_csv('/content/drive/MyDrive/졸플/BERT/Chatbot_data-master/ChatbotData.csv', encoding='utf8', low_memory=False)

In [ ]:
df_filtered = regular_df.loc[regular_df['label'] == 0]

In [ ]:
df_filtered['category_02'] = '정상'
df_filtered

<ipython-input-17-02587404148a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['category_02'] = '정상'


,Q,A,label,category_02
0,12시 땡!,하루가 또 가네요.,0,정상
1,1지망 학교 떨어졌어,위로해 드립니다.,0,정상
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,정상
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,정상
4,PPL 심하네,눈살이 찌푸려지죠.,0,정상
...,...,...,...,...
5285,힘내야지,응원합니다!,0,정상
5286,힘든 것 좀 끝났으면,다 지나갈 거예요.,0,정상
5287,힘든 시기가 지나갔으면,다 지나갈 거예요.,0,정상
5288,힘든 데도 날 도와준 사람,잊지말고 보답하세요.,0,정상


In [ ]:
regular_filtered = df_filtered[['A', 'category_02']]

In [ ]:
regular_filtered = regular_filtered.rename(columns={'A': 'note'})
regular_filtered

,note,category_02
0,하루가 또 가네요.,정상
1,위로해 드립니다.,정상
2,여행은 언제나 좋죠.,정상
3,여행은 언제나 좋죠.,정상
4,눈살이 찌푸려지죠.,정상
...,...,...
5285,응원합니다!,정상
5286,다 지나갈 거예요.,정상
5287,다 지나갈 거예요.,정상
5288,잊지말고 보답하세요.,정상


In [ ]:
regular_filtered = regular_filtered.drop_duplicates()

In [ ]:
# DataFrame에서 무작위로 200개의 행 추출
regular_filtered = regular_filtered.sample(n=500, random_state=1)

In [ ]:
regular_filtered

,note,category_02
1408,기분일꺼에요.,정상
4339,혼자 사는 것도 나쁘지 않아요.,정상
172,계속 좋지 않으면 병원에 가보세요.,정상
4864,성덕이네요.,정상
152,힘들겠네요.,정상
...,...,...
2732,아직이라면 더 살펴보세요.,정상
3799,분명 좋아할거예요.,정상
966,아이는 아이다워야 아름답죠.,정상
3229,연예인 걱정이 제일 쓸데없어요.,정상


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/졸플/BERT/abnormal_add.csv', encoding='utf8', low_memory=False)
# df = pd.read_csv('/content/drive/MyDrive/졸플/졸플/situation_data.csv', encoding='utf8', low_memory=False)

In [ ]:
del df['Unnamed: 0']

In [ ]:
df

,note,category_02
0,저는 오늘 교회에 갈 예정이에요,정상
1,우리 가족은 매년 크리스마스에 함께 파티를 해요,정상
2,나는 가끔 스포츠를 보면서 휴식을 취해요,정상
3,새로운 책을 사서 읽는 것을 즐겨해요,정상
4,우리 회사는 매주 금요일에 점심 파티를 해요.,정상
...,...,...
1000,소매치기 잡아라,절도범죄
1001,소매치기 잡아라,절도범죄
1002,저 놈 잡아라,절도범죄
1003,저 놈 잡아라,절도범죄


In [ ]:
# Load the data
df['category_02']

0         정상
1         정상
2         정상
3         정상
4         정상
        ... 
1000    절도범죄
1001    절도범죄
1002    절도범죄
1003    절도범죄
1004    폭력범죄
Name: category_02, Length: 1005, dtype: object

In [ ]:
df['category_02'] = df['category_02'].str.strip()

In [ ]:
df

,note,category_02
0,저는 오늘 교회에 갈 예정이에요,정상
1,우리 가족은 매년 크리스마스에 함께 파티를 해요,정상
2,나는 가끔 스포츠를 보면서 휴식을 취해요,정상
3,새로운 책을 사서 읽는 것을 즐겨해요,정상
4,우리 회사는 매주 금요일에 점심 파티를 해요.,정상
...,...,...
1000,소매치기 잡아라,절도범죄
1001,소매치기 잡아라,절도범죄
1002,저 놈 잡아라,절도범죄
1003,저 놈 잡아라,절도범죄


In [ ]:
# NaN 값 개수 출력
print(df.isna().sum().sum())

1


In [ ]:
# NaN 값이 있는 행 삭제
df = df.dropna()

In [ ]:
df['category_02'] = df['category_02'].replace('성폭행(성범죄)', '강제추행(성범죄)')
df['category_02'] = df['category_02'].replace('성추행(성범죄)', '강제추행(성범죄)')

<ipython-input-31-c69f75caf755>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category_02'] = df['category_02'].replace('성폭행(성범죄)', '강제추행(성범죄)')
<ipython-input-31-c69f75caf755>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category_02'] = df['category_02'].replace('성추행(성범죄)', '강제추행(성범죄)')


In [ ]:
df.to_csv('/content/drive/MyDrive/졸플/BERT/add_data.csv')

In [ ]:
data_merged = pd.concat([df, regular_filtered], axis=0, sort=False)

In [ ]:
data_merged

,note,category_02
0,저는 오늘 교회에 갈 예정이에요,정상
1,우리 가족은 매년 크리스마스에 함께 파티를 해요,정상
2,나는 가끔 스포츠를 보면서 휴식을 취해요,정상
3,새로운 책을 사서 읽는 것을 즐겨해요,정상
4,우리 회사는 매주 금요일에 점심 파티를 해요.,정상
...,...,...
2732,아직이라면 더 살펴보세요.,정상
3799,분명 좋아할거예요.,정상
966,아이는 아이다워야 아름답죠.,정상
3229,연예인 걱정이 제일 쓸데없어요.,정상


In [ ]:
data_merged.to_csv('/content/drive/MyDrive/졸플/BERT/merged_data.csv')

In [ ]:
# Encode the labels
# {"강도범죄": 0, "강제추행(성범죄)": 1, "도움요청": 2, "폭력범죄": 3, "절도범죄": 4, "정상":5}
# label_names = ['regular', 'help', 'robbery', 'sexual', 'theft', 'violence']
label_encoder = {"정상": 0, "도움요청": 1, "강도범죄": 2, "강제추행(성범죄)": 3, "절도범죄": 4, "폭력범죄":5}
data_merged['label'] = data_merged['category_02'].apply(lambda x: label_encoder[x])

In [ ]:
data_merged.columns

Index(['note', 'category_02', 'label'], dtype='object')

In [ ]:
data_merged

,note,category_02,label
0,저는 오늘 교회에 갈 예정이에요,정상,0
1,우리 가족은 매년 크리스마스에 함께 파티를 해요,정상,0
2,나는 가끔 스포츠를 보면서 휴식을 취해요,정상,0
3,새로운 책을 사서 읽는 것을 즐겨해요,정상,0
4,우리 회사는 매주 금요일에 점심 파티를 해요.,정상,0
...,...,...,...
2732,아직이라면 더 살펴보세요.,정상,0
3799,분명 좋아할거예요.,정상,0
966,아이는 아이다워야 아름답죠.,정상,0
3229,연예인 걱정이 제일 쓸데없어요.,정상,0


In [ ]:
data_merged.groupby('label').agg('count')

,note,category_02
label,,
0,561,561
1,165,165
2,256,256
3,211,211
4,138,138
5,173,173


In [ ]:
# 문자열 최대 길이 출력
max_len = data_merged['note'].str.len().max()
print(max_len)

54


In [ ]:
data_list = []
for ques, label in zip(data_merged['note'], data_merged['label'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[-800:-500]

[['이 사람이 날 죽이려고 해! 구조해줘!', '1'],
 ['도와줘! 어떻게 이 상황에서 벗어날 수 있어?', '1'],
 ['얼른 와! 제 목숨이 위협받고 있어!', '1'],
 ['구조해줘! 누군가가 나를 위협해!', '1'],
 ['도와줘! 이 상황에서 어떻게 벗어날 수 있어?', '1'],
 ['빨리 구해! 제 목숨이 위험해!', '1'],
 ['죽을래', '2'],
 ['죽고싶어', '2'],
 ['죽여버린다', '2'],
 ['죽고싶냐', '2'],
 ['짜증나네', '2'],
 ['짜증나게할래', '2'],
 ['짜증나게하지마', '2'],
 ['하지마', '3'],
 ['손대지마', '3'],
 ['만지지마', '3'],
 ['하지마라', '3'],
 ['현금내놔', '2'],
 ['금고열어', '2'],
 ['금고어딨어', '2'],
 ['어딜만져', '3'],
 ['귀금속내놔', '2'],
 ['귀중품내놔', '2'],
 ['귀중품어딨어', '2'],
 ['비밀번호뭐야', '2'],
 ['어딜손대', '3'],
 ['내몸만지지마', '3'],
 ['만지지마세요', '3'],
 ['내몸에손대지마', '3'],
 ['비밀번호뭐냐니까', '2'],
 ['하지말라고', '3'],
 ['귀금속어디다뒀어', '2'],
 ['내 몸 만지지 마', '3'],
 ['입막아', '2'],
 ['돈 내놔', '2'],
 ['왜 그러세요', '3'],
 ['빨리 안 내놔', '2'],
 ['지갑 내놔', '2'],
 ['나한테 왜 이래', '3'],
 ['가진거 다 내놔', '2'],
 ['제발 하지 마세요', '3'],
 ['저한테 왜 이러세요', '3'],
 ['하지 마세요', '3'],
 ['하지 말란 말이야', '3'],
 ['하지마 하지 말란 말이야', '3'],
 ['저한테 왜 그러시는데요', '3'],
 ['왜 왜 이러세요', '3'],
 ['나한테 왜 그래', '3'],
 ['꼼짝마', '2'],
 ['조용히 해', '2'],
 ['입 다물어',

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 60
batch_size = 64
warmup_ratio = 0.1
num_epochs = 12  # 10번 이상
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
dataset_train[:10]

[['어디에 갔든 돈을 내놓아야 합니다. 그렇지 않으면 안 됩니다.', '2'],
 ['장갑을 끼세요.', '0'],
 ['저는 도움이 되고 싶어요.', '0'],
 ['시원한 바람 쉬고 약먹기!', '0'],
 ['제발, 그만해주세요! 이게 성범죄예요!', '3'],
 ['이걸로 너의 빚을 갚았다 생각해.', '5'],
 ['도와 주세요 ', '1'],
 ['내가 니가 좋아하는 거 다 알아. 그러니까 말하지 않아도 돼.', '3'],
 ['죽고싶냐', '5'],
 ['가족들과 함께 하루 종일 근처 쇼핑몰을 돌아다니면서 쇼핑을 할 예정이에요.', '0']]

In [ ]:
tok=kobert_tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.rand(10).cuda()

tensor([0.2741, 0.5431, 0.1195, 0.7387, 0.0045, 0.9315, 0.0373, 0.9070, 0.4200,
        0.4605], device='cuda:0')

### Finetuning 진행

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
y_true, y_pred = [], []
for e in range(num_epochs):
    
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        _, y_pred_label = torch.max(out, dim=1)
        y_true.extend(label.cpu().numpy())
        y_pred.extend(y_pred_label.cpu().numpy())

        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/19 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.08782010525465012 train acc 0.96875
epoch 1 train acc 0.990953947368421


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 1 test acc 0.9460416666666667


...

epoch 11 test acc 0.9478472222222223


  0%|          | 0/19 [00:00<?, ?it/s]

epoch 12 batch id 1 loss 0.07302948087453842 train acc 0.984375
epoch 12 train acc 0.9868421052631579


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 12 test acc 0.9366666666666668


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)

In [ ]:
cm

array([[1307,    0,    8,    5,    0,    0],
       [   0,  369,    0,    8,    2,   17],
       [   0,    0,  568,   16,    9,    7],
       [   0,    0,    0,  420,    0,   12],
       [   0,    0,    0,    8,  363,    1],
       [   7,   22,   22,   67,   12,  362]])

In [ ]:
print(f"Training accuracy: {train_history[-1]*100:.2f}%, Validation accuracy: {test_history[-1]*100:.2f}%")

Training accuracy: 98.68%, Validation accuracy: 93.67%


In [ ]:
from sklearn.metrics import f1_score

# Compute the F1 score
f1 = f1_score(y_true, y_pred, average='macro')

print('F1 score: %.2f' % f1)

F1 score: 0.92


In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize the correct and total predictions counters
correct_predictions = 0
total_predictions = 0
y_true, y_pred = [], []

# Loop over the validation data loader
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    # Move the inputs and labels to the device
    token_ids = token_ids.long().to(device)
    valid_length = valid_length.to(device)
    segment_ids = segment_ids.long().to(device)
    label = label.to(device)

    # Compute the predicted outputs
    with torch.no_grad():
        outputs = model(token_ids, valid_length, segment_ids)

    # Get the predicted labels
    _, predicted_labels = torch.max(outputs.data, 1)
    y_true.extend(label.cpu().numpy())
    y_pred.extend(predicted_labels.cpu().numpy())

    # Update the counters
    total_predictions += label.size(0)
    correct_predictions += (predicted_labels == label).sum().item()

    # Compute the validation accuracy
    val_accuracy = 100.0 * correct_predictions / total_predictions

print(correct_predictions)
print(total_predictions)
print('Validation accuracy: %.2f %%' % val_accuracy)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


282
301
Validation accuracy: 93.69 %


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
cm

array([[110,   0,   0,   0,   0,   0],
       [  0,  31,   0,   1,   0,   1],
       [  0,   0,  45,   3,   1,   1],
       [  0,   0,   0,  35,   0,   1],
       [  0,   0,   0,   1,  29,   1],
       [  0,   3,   0,   5,   1,  32]])

In [ ]:
from sklearn.metrics import f1_score

# Compute the F1 score
f1 = f1_score(y_true, y_pred, average='macro')

print('F1 score: %.2f' % f1)

F1 score: 0.92


### model 저장

In [ ]:
# 모델 저장
PATH = '/content/drive/MyDrive/졸플/BERT'

torch.save(model, PATH + 'kobert_model_4.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

# Kobert test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## 학습 모델 로드
PATH = '/content/drive/MyDrive/졸플/BERT'
model = torch.load(PATH + 'kobert_model_4.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

<All keys matched successfully>

In [ ]:
PATH = '/content/drive/MyDrive/졸플/BERT'
model = BERTClassifier(bertmodel, dr_rate=0.5)
device = torch.device('cpu')
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt', map_location=device))


<All keys matched successfully>

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-a7u75iv0
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-a7u75iv0
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
# Setting parameters
max_len = 60
batch_size = 64
warmup_ratio = 0.1
num_epochs = 8  # 3번으로도 충분해보인다
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
# from tqdm import tqdm, tqdm_notebook
import time

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
# device = torch.device('cpu')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# model = torch.load('/content/drive/MyDrive/졸플/BERT/kobert_model.pt') # class = 5 without 정상
# model = torch.load('/content/drive/MyDrive/졸플/BERT/kobert_model_3.pt')  # class = 6, include 정상

In [ ]:
#BERT 모델, Vocabulary 불러오기
from transformers import AutoModel, AutoTokenizer

from kobert import get_pytorch_kobert_model

kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)
# kobert = AutoModel.from_pretrained("skt/kobert-base-v1")
bertmodel, vocab = get_pytorch_kobert_model()

tok=kobert_tokenizer.tokenize


Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
def text_predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        logit_list = []
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            logit_list.append(logits)

            # label_encoder = {"정상": 0, "도움요청": 1, "강도범죄": 2, "강제추행(성범죄)": 3, "절도범죄": 4, "폭력범죄":5}
            if np.argmax(logits) == 0:
                test_eval.append("정상")
            elif np.argmax(logits) == 1:
                test_eval.append("도움요청")
            elif np.argmax(logits) == 2:
                test_eval.append("강도범죄")
            elif np.argmax(logits) == 3:
                test_eval.append("강제추행(성범죄)")
            elif np.argmax(logits) == 4:
                test_eval.append("절도범죄")
            elif np.argmax(logits) == 5:
                test_eval.append("폭력범죄")


        # print(">> 입력하신 내용은 " + test_eval[0])
        return test_eval[0] , logit_list[0], np.argmax(logit_list[0])

In [ ]:
import time

startTime = time.time()

# model = torch.load('/content/drive/MyDrive/졸플/BERT/kobert_model_3.pt') 
# device = torch.device('cpu')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
# model.to('cpu')

print(text_predict("만지지 마세요. 저리 가세요."))

endTime = time.time() - startTime
print('걸린 시간 :',  endTime)

# mfcc -> 성능이 어느정도 향상 
# 모델 
# 데이터를 더 추가하거나
# 데이터 aug X
# 데이터 타입을 바꾸기 
# melspec
# mfcc

# kobert 몇 초 나오는지 측정
# kobert gpu -> 

RuntimeError: ignored

In [ ]:
text_predict("만지지 마세요. 저리 가세요.")

('강제추행(성범죄)',
 array([ 0.23792514, -1.2807266 , -1.4002788 ,  4.0378475 , -0.02669628,
        -1.4447587 ], dtype=float32),
 3)

In [ ]:
text_predict("안돼요. 만지지 마세요")

('강제추행(성범죄)',
 array([-0.521649 , -0.6714706, -1.0557039,  4.289683 , -0.6412717,
        -1.1418635], dtype=float32),
 3)

In [ ]:
text_predict("아파요! 이 사람이 나 폭행하고 있어요!")

('강제추행(성범죄)',
 array([-0.947373  , -0.6130026 , -1.210949  ,  4.43949   , -0.64738256,
        -0.7637912 ], dtype=float32),
 3)

In [ ]:
text_predict("도둑이야! 저 사람 좀 잡아주세요!")

('절도범죄',
 array([-0.814848  , -0.56955314, -1.141165  , -0.41330785,  4.135758  ,
        -0.94977754], dtype=float32),
 4)

In [ ]:
text_predict("도둑이야! 내 돈이 사라졌어")

('절도범죄',
 array([-0.8354762 , -0.9383715 , -0.8863815 , -0.58790326,  4.210293  ,
        -0.6653168 ], dtype=float32),
 4)

In [ ]:
text_predict("앗, 저 사람 좀 잡아주세요! 제 돈을 가져갔어요!")

('절도범죄',
 array([-1.314539  , -0.3519858 , -1.1553864 , -0.4322144 ,  4.0706906 ,
        -0.49031955], dtype=float32),
 4)

In [ ]:
text_predict("날씨가 좋네요!")

('정상', array([ 5.4743757 , -0.98013055, -0.6948316 , -1.1269101 , -0.46413082,
        -2.0242088 ], dtype=float32), 0)

In [ ]:
text_predict("당신을 사랑해요")

('정상', array([ 4.4296017, -1.0681657, -0.3009089, -0.5937653, -1.1321213,
        -1.2383188], dtype=float32), 0)